In [1]:
import mne
from netCDF4 import Dataset
import json
import dask.array as da
from dask import delayed
import os, logging
from services.utils.timing import TimingContext
import pyarrow as pa
from services.delta_lake import Duck_Lake
from prefect import flow, task
from prefect_dask import DaskTaskRunner
from dataclasses import dataclass, asdict
from typing import List

logging.basicConfig()
logging.root.setLevel(logging.INFO)

ducklake = Duck_Lake()

my_edf_file_path = os.path.join(
    os.environ["CONTAINER_FILE_STORAGE_PATH"],
    "test33_HypoactiveHeidi_05_DAY1_PROCESSED.edf",
)
my_parquet_output_dir = os.path.join(os.environ["CONTAINER_FILE_STORAGE_PATH"], "test")

misc_channels = [
    "pitch",
    "roll",
    "heading",
    "GyrZ",
    "MagZ",
    "Tag_On",
    "Depth",
    "MagX",
    "MagY",
]

@dataclass
class SignalSchema:
    signal_name: str
    frequency: float
    start_time: float
    data: List[float]

@task
def read_signal(
    edf_file_path,
    signal_name,
):
    """Function to read a single signal from an EDF file."""
    raw = mne.io.read_raw_edf(edf_file_path, preload=False)
    signal = raw.pick(signal_name).get_data()
    
    return SignalSchema(
        signal_name=signal_name,
        frequency=raw.info["sfreq"],
        start_time=pa.scalar(raw.info["meas_date"].timestamp(), type=pa.timestamp('us', tz="UTC")),
        # start_time=raw.info["meas_date"],
        data=signal[0],
    )


@flow
def process_edf(
    edf_file_path: str,
    schema: pa.schema,
    misc_channels: List[str] = misc_channels,
):
    with TimingContext("EDF Read"):
        raw = mne.io.read_raw_edf(edf_file_path, preload=False)

        channel_types = dict()
        
        for k in raw.ch_names:
            if k in misc_channels:
                channel_types[k] = "misc"
            else:
                channel_types[k] = "eeg"
        raw.set_channel_types(channel_types)
        
        channels_to_use = [ch for ch in raw.ch_names if ch not in misc_channels]
        
        buff = []
        for signal_name in channels_to_use:
            signal = read_signal(edf_file_path, signal_name)
            buff.append(asdict(signal))
        
        _ = buff
        table = pa.Table.from_pylist(buff, schema=schema)
        
        ducklake.write_to_delta(
            data=table,
            schema=schema,
            mode="append",
            partition_by=['signal_name'],
            name="test",
            description="test"
        )


schema = pa.schema(
    [
        pa.field("signal_name", pa.string()),
        pa.field("frequency", pa.float64()),
        pa.field("start_time", pa.timestamp('us', tz="UTC")),
        pa.field("data", pa.list_(pa.float64())),
    ]
)

with TimingContext("Main"):
    process_edf(my_edf_file_path, schema)


/usr/local/lib/python3.12/site-packages/pydantic/_internal/_generate_schema.py:404: UserWarning: <cyfunction schema at 0xffff294bfb90> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(
/usr/local/lib/python3.12/site-packages/pydantic/_internal/_fields.py:201: UserWarning: Field name "schema" in "ProcessEdf" shadows an attribute in parent "V2ValidatedFunction.create_model.<locals>.DecoratorBaseModel"
  warnings.warn(


01:21:27.319 | INFO    | root - Starting Main...

/usr/local/lib/python3.12/site-packages/pydantic/_internal/_fields.py:201: UserWarning: Field name "schema" in "ProcessEdf" shadows an attribute in parent "V2ValidatedFunction.create_model.<locals>.DecoratorBaseModel"
  warnings.warn(
/usr/local/lib/python3.12/site-packages/pydantic/_internal/_generate_schema.py:404: UserWarning: <cyfunction schema at 0xffff294bfb90> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(


01:21:27.430 | INFO    | prefect.engine - Created flow run 'hidden-lemming' for flow 'process-edf'

01:21:27.431 | INFO    | Flow run 'hidden-lemming' - View at http://prefect:4200/flow-runs/flow-run/64ce5d9e-e047-4573-beb0-c926f5258b8b

...

01:23:56.116 | INFO    | distributed.core - Event loop was unresponsive in Nanny for 24.19s.  This is often caused by long-running GIL-holding functions or moving large chunks of data. This can cause timeouts and instability.

01:23:56.232 | INFO    | distributed.core - Event loop was unresponsive in Nanny for 24.31s.  This is often caused by long-running GIL-holding functions or moving large chunks of data. This can cause timeouts and instability.

: 

: 

: 